In [0]:
# 00_config_auth — autenticação via SAS guardado no Secret Scope (job-safe)

# --- parâmetros (podem ser sobrescritos pelo Job via "Parameters") ---
# Se preferir, edite aqui os valores padrão.
try:
    dbutils.widgets.get               # garante que widgets existem
except:
    pass

def _wget(name, default):
    try:
        dbutils.widgets.text(name, default)
    except:
        # widget já existia
        pass
    return dbutils.widgets.get(name)

acc   = _wget("storage_account", "stgflowdev001")        # sem .dfs
scope = _wget("secret_scope",    "meu_scope")
key   = _wget("secret_key",      "sas-stgflowdev001")

# --- lê o SAS do Secret Scope ---
try:
    sas_raw = dbutils.secrets.get(scope, key)
except Exception as e:
    raise Exception(
        f"[Auth] Falha ao ler secret: scope='{scope}', key='{key}'. "
        "Verifique se o scope e a chave existem e se você tem permissão."
    ) from e

# --- limpa o SAS (remove quebras de linha/espaços e '?' inicial, se houver) ---
sas = sas_raw.strip().replace("\n", "").replace("\r", "")
if sas.startswith("?"):
    sas = sas[1:]

if not sas.startswith("sv="):
    raise ValueError(
        "[Auth] O valor do SAS não parece válido (não inicia com 'sv='). "
        "Confira o segredo salvo no scope."
    )

# --- configura o Spark para usar SAS ---
spark.conf.set(f"fs.azure.account.auth.type.{acc}.dfs.core.windows.net", "SAS")
spark.conf.set(f"fs.azure.sas.token.provider.type.{acc}.dfs.core.windows.net",
               "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set(f"fs.azure.sas.fixed.token.{acc}.dfs.core.windows.net", sas)

# --- caminhos base (para usar nas células seguintes) ---
bronze_base = f"abfss://bronze@{acc}.dfs.core.windows.net"
silver_base = f"abfss://silver@{acc}.dfs.core.windows.net"
gold_base   = f"abfss://gold@{acc}.dfs.core.windows.net"

print("[Auth] OK — SAS aplicado e caminhos definidos.")


In [0]:
import requests, json, time
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

bronze_base       = f"abfss://bronze@{acc}.dfs.core.windows.net"
bronze_delta_path = f"{bronze_base}/fx_usdbrl/delta"

def fetch_usd_brl():
    # 1) Frankfurter
    try:
        r = requests.get("https://api.frankfurter.app/latest?from=USD&to=BRL", timeout=10)
        r.raise_for_status()
        p = r.json()
        rate = float(p["rates"]["BRL"])
        ref_date = p.get("date")
        base = p.get("base", "USD")
        return {"provider":"frankfurter", "base":base, "rate":rate, "ref_date":ref_date, "raw":p}
    except Exception:
        pass
    # 2) Open ER-API
    r = requests.get("https://open.er-api.com/v6/latest/USD", timeout=10)
    r.raise_for_status()
    p = r.json()
    if p.get("result") != "success":
        raise ValueError(f"Provider ER-API error: {p}")
    rate = float(p["rates"]["BRL"])
    ref_date = p.get("time_last_update_utc")  # timestamp amigável
    return {"provider":"er-api", "base":"USD", "rate":rate, "ref_date":ref_date, "raw":p}

data = fetch_usd_brl()
assert data["rate"] > 0, "Taxa inválida"

record = [{
    "provider": data["provider"],
    "base":     data["base"],
    "symbol":   "BRL",
    "rate":     data["rate"],
    "ref_date": data["ref_date"],
    "ingestion_epoch": int(time.time()),
    "raw_json": json.dumps(data["raw"], ensure_ascii=False)
}]

schema = StructType([
    StructField("provider", StringType(), True),
    StructField("base", StringType(), True),
    StructField("symbol", StringType(), True),
    StructField("rate", DoubleType(), True),
    StructField("ref_date", StringType(), True),
    StructField("ingestion_epoch", StringType(), True),
    StructField("raw_json", StringType(), True),
])

df_bronze = spark.createDataFrame(record, schema)\
                 .withColumn("ingestion_ts", F.current_timestamp())

(df_bronze
 .write
 .format("delta")
 .mode("append")
 .option('mergeSchema', 'true').save(bronze_delta_path)
)

display(spark.read.format("delta").load(bronze_delta_path).orderBy(F.col("ingestion_ts").desc()))


In [0]:
import requests, json, time
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType

# 1) Consulta HTTP (sem token)
url = "https://api.exchangerate.host/latest?base=USD&symbols=BRL"
resp = requests.get(url, timeout=10)
resp.raise_for_status()
payload = resp.json()

# 2) Extrai campos principais
# Exemplo de payload:
# {"base":"USD","date":"2025-08-19","rates":{"BRL":5.27}, "success":true}
base = payload.get("base")
date = payload.get("date")
rate = float(payload.get("rates", {}).get("BRL", 0.0))
ingestion_ts = int(time.time())

record = [{
    "base": base,
    "symbol": "BRL",
    "rate": rate,
    "ref_date": date,          # data de referência informada pela API
    "ingestion_epoch": ingestion_ts,  # carimbo da coleta
    "raw_json": json.dumps(payload, ensure_ascii=False)  # trilha "raw"
}]

schema = StructType([
    StructField("base", StringType(), True),
    StructField("symbol", StringType(), True),
    StructField("rate", DoubleType(), True),
    StructField("ref_date", StringType(), True),
    StructField("ingestion_epoch",  StringType(), True),
    StructField("raw_json", StringType(), True),
])

df_bronze = spark.createDataFrame(record, schema)\
                 .withColumn("ingestion_ts", F.current_timestamp())

# 3) Escreve/Anexa em Delta (Bronze)
(
    df_bronze
    .write
    .format("delta")
    .mode("append")
    .save(bronze_delta_path)
)

print("Bronze ✅ gravado em:", bronze_delta_path)
display(spark.read.format("delta").load(bronze_delta_path).orderBy(F.col("ingestion_ts").desc()))
